In [1]:
import numpy as np
import pandas as pd
#!pip install anndata --target D:/usr/local/lib/python3.8/dist-packages
import anndata as ad
from collections import defaultdict
from rnasieve.preprocessing import model_from_raw_counts
print(ad.__version__)

0.7.8


This file uses guidance from https://github.com/theislab/anndata-tutorials/blob/master/getting-started.ipynb
 to get the jcoffman008 data as an AnnData structure so that it can be used for RNAsieve as in https://github.com/songlab-cal/rna-sieve


In [5]:
##-----loading in jcoffman007 expression and design data -----
#load in jcoffman007 count data (raw counts)
widejcoffman007_count = pd.read_csv("all_bulk_count_file.csv.gz", compression="gzip")
#changing GeneID_genename format to just GeneID
# widejcoffman007_count["gene_id"]=widejcoffman007_count["gene_id"].str[:18]
#load in design file. note that each row is a sample that matches up with the "data" dataframe  
jcoffman007_design= pd.read_csv("all_bulk_design_file.csv.gz", compression="gzip",index_col=0)


##-----loading in atlas expression and design data----- 
#uploading the expression data for the reference (atlas)
wideatlas_count = pd.read_csv("fill_sorted_matched_Hou_shared_gene_counts.csv.gz", compression="gzip")
#changing GeneID|genename format to just GeneID
# wideatlas_count["gene"]=wideatlas_count["gene"].str[:18]
#uploading the meta/"obs" for the reference (atlas)
''' 
atlas_meta_alldpf = pd.read_csv("meta.tsv", sep="\t",index_col=0)
#restricting meta to only 5dpf 
sample_names_5dpf= ['5a_5dpf', '5b_5dpf']
atlas_meta= atlas_meta_alldpf[atlas_meta_alldpf['sample_name'].isin(sample_names_5dpf)]
'''
atlas_design = pd.read_csv("sorted_labeled_GSE137971_cells.csv.gz", compression="gzip",index_col=0)


In [6]:
#altering both expression datasets to contain identical sets of genes 
jcoffman007_genes= widejcoffman007_count["genes"].tolist()
atlas_genes= wideatlas_count["genes"].tolist()
genes_in_both= list(set(jcoffman007_genes) & set(atlas_genes))
print("there are",len(genes_in_both),"genes that are in both the atlas and jcoffman007 count data") 
widejcoffman007_count=widejcoffman007_count[widejcoffman007_count["genes"].isin(genes_in_both)]
wideatlas_count=wideatlas_count[wideatlas_count["genes"].isin(genes_in_both)]
#sorting to be in same order
widejcoffman007_count=widejcoffman007_count.sort_values(by=['genes'])
wideatlas_count=wideatlas_count.sort_values(by=['genes'])

there are 24324 genes that are in both the atlas and jcoffman007 count data


In [7]:
#------making annData structure that holds the counts and meta for jcoffman007-----
#convert to having genes as columns, samples as rows (first have to make the gene ID the row label)
widejcoffman007_count= widejcoffman007_count.set_index("genes")
jcoffman007_count= widejcoffman007_count.transpose()
jcoffman007 = ad.AnnData(jcoffman007_count, obs=jcoffman007_design)

#-----making annData structure that holds the counts and meta for atlas----- 
wideatlas_count=wideatlas_count.set_index("genes")
atlas_count= wideatlas_count.transpose()
atlas = ad.AnnData(atlas_count, obs=atlas_design)

In [8]:
widejcoffman007_count

,SL139775,SL139776,SL139777,SL139781,SL139782,SL139783,SL139787,SL139788,SL139789,SL139772,SL139773,SL139774,SL139778,SL139779,SL139780,SL139784,SL139785,SL139786
genes,,,,,,,,,,,,,,,,,,
ENSDARG00000000001,304.595549,373.334980,315.167217,265.285938,254.996703,294.484331,288.226016,274.347695,298.029937,404.056268,365.712681,348.952161,360.809047,323.567065,297.090172,330.758477,291.916034,335.044507
ENSDARG00000000002,218.404451,323.665020,321.832876,3169.714090,2648.003297,3220.527799,2109.773984,1063.652305,2008.970759,631.954651,573.365921,615.382265,2571.190953,2599.433115,2772.922784,1910.241523,1787.114720,1869.955493
ENSDARG00000000018,493.000000,681.000000,617.000000,1209.000000,1070.000000,1263.000000,1163.000000,965.000000,1191.000000,1052.000000,901.000000,843.000000,1124.000000,1053.000000,933.000000,1199.000000,1285.000000,1089.000000
ENSDARG00000000019,1677.378292,2553.272563,2512.293841,5922.827011,5163.483274,6213.939414,2433.553081,2150.427606,2828.578269,4270.180122,4251.043771,3804.370723,6233.160439,5889.509691,5069.502014,4593.545358,4841.388083,3790.898820
ENSDARG00000000068,755.000000,845.000000,807.000000,1028.000000,922.000000,997.000000,649.000000,653.000000,828.000000,632.000000,616.000000,585.000000,853.000000,834.000000,815.000000,680.000000,643.000000,666.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSDARG00000109183,38.775380,62.049028,57.808283,53.101703,58.272741,54.579709,95.957037,72.129155,82.569700,66.403613,74.831938,62.874552,63.276487,64.872432,57.510786,71.922886,83.536854,52.526739
ENSDARG00000109184,132.944753,142.429683,120.719244,216.116346,178.069070,187.834436,209.844739,141.197546,141.152659,201.437297,164.410399,162.134992,134.107153,115.170918,125.614306,144.523689,120.608478,112.967306
ENSDARG00000109185,202.424643,232.219174,157.950006,72.579199,37.601526,59.614848,259.360619,184.232489,132.186459,35.009017,28.338946,60.838462,34.995281,49.684690,31.027428,16.002811,12.312602,46.250170


In [9]:
#modeling off of the raw counts prep section from example.ipynb from song lab github

# grouping the cells in the reference data by cluster 
print('Aggregating by cluster...')
counts_by_cluster = {}
for i in range(len(atlas)):
    sc = atlas[i]
    if len(sc.obs['major.cl']) == 0:
        continue
    cell_cluster = sc.obs['major.cl'][0]
    if cell_cluster not in counts_by_cluster:
        counts_by_cluster[cell_cluster] = np.empty((sc.X.shape[1], 0), dtype=np.float32)
    counts_by_cluster[cell_cluster] = np.hstack(
        (counts_by_cluster[cell_cluster], sc.X.toarray().reshape(-1, 1)))
    

# Bulk prep
print('Aggregating bulks by name...')
G = jcoffman007.n_vars
bulk_by_time = defaultdict(list)
for i in range(len(jcoffman007)):
    bulk = jcoffman007[i]
    if len(bulk.obs['name']) == 0:
        continue
    time = bulk.obs['name'][0]
    bulk_by_time[time].append(bulk.X.toarray().reshape(-1, 1))

bulk_labels = []
psis = np.empty((G, 0), dtype=np.float32)
for name in sorted(bulk_by_time.keys()):
    bulks = bulk_by_time[name]
    for i in range(len(bulks)):
        bulk_labels.append("{} name, subject {}".format(name, i))
        psis = np.hstack((psis, bulks[i]))
        
print('Done!')

Aggregating by cluster...
Aggregating bulks by name...
Done!


In [10]:
counts_by_cluster['Intermediate Epithelial'].shape

(24324, 5836)

In [11]:
model, cleaned_psis = model_from_raw_counts(counts_by_cluster, psis[:, :18])

In [12]:
cleaned_psis

,Bulk 0,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15,Bulk 16,Bulk 17
0,218.404449,323.665009,321.832886,3169.714111,2648.003418,3220.527832,2109.773926,1063.652344,2008.970703,631.954651,573.365906,615.382263,2571.190918,2599.433105,2772.922852,1910.241577,1787.114746,1869.955444
1,755.000000,845.000000,807.000000,1028.000000,922.000000,997.000000,649.000000,653.000000,828.000000,632.000000,616.000000,585.000000,853.000000,834.000000,815.000000,680.000000,643.000000,666.000000
2,1970.340454,2339.308350,2001.430786,3111.187256,2748.340088,3177.886719,3795.539307,3543.188232,3726.367188,1795.925537,1695.094971,1565.693604,3509.776855,3487.701904,2715.402588,3590.147705,3809.764160,3905.987305
3,56411.500000,52280.015625,64635.925781,173177.500000,162114.765625,183359.062500,24562.027344,27122.585938,29532.625000,145650.125000,146573.468750,128260.828125,158043.296875,128596.046875,121123.156250,46315.144531,49794.390625,48282.339844
4,2960.000000,3706.000000,3667.000000,5214.000000,4559.000000,5547.000000,3692.000000,2993.000000,3664.000000,4230.000000,4282.000000,3993.000000,5520.000000,5189.000000,4595.000000,3676.000000,4086.000000,3835.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5501,530.809448,500.286652,729.000000,1066.024170,896.637329,1153.005005,503.248199,558.282410,536.533752,6539.441895,5886.029785,5865.681641,683.286194,731.824158,419.000000,3352.666748,3015.272949,1879.553467
5502,269.000000,295.000000,330.000000,421.000000,348.000000,452.000000,344.000000,275.000000,304.000000,162.000000,156.000000,191.000000,219.000000,225.000000,234.000000,119.000000,164.000000,241.000000
5503,20105.685547,35265.863281,18700.068359,7920.672363,5529.533691,7514.861816,17459.433594,7870.057617,11492.776367,25168.390625,35221.781250,31393.162109,5661.808105,6905.750977,5512.356445,10270.350586,10393.014648,9425.689453
5504,838.482666,1093.448975,1001.469543,921.379761,827.557495,966.561523,1060.946777,1000.886414,1333.324097,1020.471436,1022.944824,1105.269653,810.771240,815.942383,760.369873,1090.449951,1124.683105,1085.885254


In [13]:
output_table= model.predict(cleaned_psis) #this takes a minute ... 
output_table.to_csv('resorted_rnasieve_jcoffman007.csv')

In [14]:
# In this example, the intervals at a significance level of 0.05 indicate the estimate is poor.
# We set sig=0.9999 for the sake of visualization.
model.compute_marginal_confidence_intervals(sig=0.2)

[[(0.1307589642566624, 0.18818826753466428),
  (0.04548098285277577, 0.07186437497602365),
  (0.0, 0.03570127765552664),
  (0.03096124963299844, 0.04785526377705264),
  (0.05244835506723965, 0.1386440674427165),
  (0.6101114324631146, 0.6554902741568635)],
 [(0.152573825052785, 0.20454271842012212),
  (0.04902525578349883, 0.07150713573161759),
  (0.0, 0.017557897665230037),
  (0.04644565308772227, 0.06540415796945183),
  (0.06872453550741604, 0.14978378756441046),
  (0.5714752410496672, 0.6205176898333088)],
 [(0.14098535533883005, 0.19787743490728993),
  (0.04297705326518207, 0.0656979980175854),
  (0.0, 0.01894208491716336),
  (0.03490805208011475, 0.05162408999944013),
  (0.048227758713977976, 0.12394741991088123),
  (0.6241316800788393, 0.6696231576878592)],
 [(0.49629001987774823, 0.5453804032335017),
  (0.012777741483354551, 0.020116067992546678),
  (0.0, 0.004269430950781408),
  (0.17425076619525004, 0.2028316868552421),
  (0.021878430268103974, 0.04246319829552833),
  (0.22380

In [15]:
model.plot_proportions('bar').properties(title="Bar visualization ")

alt.FacetChart(...)

In [16]:
model.plot_proportions('stacked').properties(title="Stacked visualization")

alt.Chart(...)